### Salaries Dataset Cleaning and Aggregation

##### This notebooks highlights case study analysis on salaries dataset gathered from Kaggle, and will be used to highlight:
- Data Cleaning Techniques
- Data Aggregations and Pivot Tables
- Exploratory Data Analysis (EDA)

In [124]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

##### Simple 'ETL' Pipeline Class for the Dataset

In [125]:
# a basic data pipeline class that loads and deletes empty cols
class Pipeline:

    def __init__(self) -> None:
        self.data = None
    
    # loads csv file
    def load_csv(self, src: str):
        
        try:
            self.data = pd.read_csv(src, low_memory=False)
            return self.data
        except ValueError as e:
            return "The DataFrame is empty."

    # drops empty columns
    def drop_cols(self, cols: list):
        self.data = self.data.drop(columns=cols, axis=1)
        return self.data

    # drops specific rows
    def drop_rows(self, row: list):
        self.data = self.data.drop(rows=row)
        return self.data
    
    # fill values
    def fill_values(self):
        self.data.fillna(0, inplace=True)
        return self.data
    
    def drop_missing(self):
        self.data = self.data.dropna()
        return self.data

In [126]:
data = Pipeline()

df = data.load_csv("..\data\Salaries.csv")

In [127]:
df.columns

Index(['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay', 'OtherPay',
       'Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Notes', 'Agency',
       'Status'],
      dtype='object')

In [128]:
df.shape

(148654, 13)

In [129]:
# renaming columns
headers = [
    "id", "employee-name", "job-title", "base-pay", "overtime-pay", "other-pay",
    "benefits", "total-pay", "total-pay-benefits", "year", "notes", "agency", "status"
]

df.columns = headers

In [130]:
# dropping unnecessary/empty columns
df = data.drop_cols(['agency', 'notes', 'status'])
df = data.fill_values()

df.loc[:, 'base-pay':'benefits'].isna().sum()

base-pay        0
overtime-pay    0
other-pay       0
benefits        0
dtype: int64

In [131]:
df.dtypes

id                      int64
employee-name          object
job-title              object
base-pay               object
overtime-pay           object
other-pay              object
benefits               object
total-pay             float64
total-pay-benefits    float64
year                    int64
dtype: object

In [132]:
# converting numerical object types into float data type
df = df[~df.astype(str).apply(lambda row: row.str.contains("Not provided", case=False).any(), axis=1)].copy()

df[['base-pay', 'overtime-pay', 'other-pay', 'benefits']] = df[['base-pay', 'overtime-pay', 'other-pay', 'benefits']].astype(float)

# verify data types
df.dtypes

id                      int64
employee-name          object
job-title              object
base-pay              float64
overtime-pay          float64
other-pay             float64
benefits              float64
total-pay             float64
total-pay-benefits    float64
year                    int64
dtype: object

In [133]:
# subsetting the dataframe with numerical values to remove 0 or negative values
df = df[(df.loc[:, "base-pay":"total-pay-benefits"] >= 0).all(axis=1)]

# dropping the 0 values on total-pay as we treat them as not provided
df = df[df['total-pay'] > 0]

# lambda function to standardize string values
df.loc[:, "employee-name":"job-title"] = df.loc[:, "employee-name":"job-title"].apply(lambda cap: cap.str.title())

# counting values that still contains "Not Provided"
count = (df.loc[:, "employee-name":"job-title"] == "Not Provided").any(axis=1).sum()

print(count)

0


In [134]:
# formatting job title strings
df['job-title'] = df['job-title'].str.replace(r'\bIii\b', 'III', regex=True)
df['job-title'] = df['job-title'].str.replace(r'\bIi\b', 'II', regex=True)
df['job-title'] = df['job-title'].str.replace(r'\bIv\b', 'IV', regex=True)

df

,id,employee-name,job-title,base-pay,overtime-pay,other-pay,benefits,total-pay,total-pay-benefits,year
0,1,Nathaniel Ford,General Manager-Metropolitan Transit Authority,167411.18,0.00,400184.25,0.00,567595.43,567595.43,2011
1,2,Gary Jimenez,Captain III (Police Department),155966.02,245131.88,137811.38,0.00,538909.28,538909.28,2011
2,3,Albert Pardini,Captain III (Police Department),212739.13,106088.18,16452.60,0.00,335279.91,335279.91,2011
3,4,Christopher Chong,Wire Rope Cable Maintenance Mechanic,77916.00,56120.71,198306.90,0.00,332343.61,332343.61,2011
4,5,Patrick Gardner,"Deputy Chief Of Department,(Fire Department)",134401.60,9737.00,182234.59,0.00,326373.19,326373.19,2011
...,...,...,...,...,...,...,...,...,...,...
148621,148622,Kenneth Mackey,Transit Operator,0.00,0.00,15.35,0.00,15.35,15.35,2014
148622,148623,Jacqueline Hubbard,Senior Clerk,0.00,0.00,13.77,0.00,13.77,13.77,2014
148623,148624,Deborah B Honig,Attorney (Civil/Criminal),0.00,0.00,13.35,0.13,13.35,13.48,2014
148624,148625,Lorraine Rosenthal,Senior Clerk,0.00,0.00,12.89,0.00,12.89,12.89,2014


In [135]:
# summary stats check on the curr. dataframe
df.describe().round(2)

,id,base-pay,overtime-pay,other-pay,benefits,total-pay,total-pay-benefits,year
count,148266.00,148266.00,148266.00,148266.00,148266.00,148266.00,148266.00,148266.00
mean,74239.00,66223.56,5078.85,3658.14,18956.17,74960.54,93916.71,2012.52
std,42921.43,42811.29,11466.13,8064.84,17175.13,50440.14,62718.75,1.12
min,1.00,0.00,0.00,0.00,0.00,0.30,0.30,2011.00
25%,37071.25,33297.32,0.00,0.00,0.00,36690.87,44716.52,2012.00
50%,74174.50,64957.53,0.00,819.59,23376.78,71555.94,92577.83,2013.00
75%,111533.75,94691.01,4680.88,4255.35,33493.14,105967.10,133008.42,2014.00
max,148626.00,319275.01,245131.88,400184.25,96570.66,567595.43,567595.43,2014.00


##### Load the Cleaned DataFrame into CSV File

In [136]:
# load csv into
df.to_csv("..\data\salaries_cleaned.csv", index=False)

#### Data Aggregation

In [137]:
# average base pay for all employees/occupation per year
avg_base_pay = df.groupby('year').agg({'base-pay' : 'mean'}).round(2)

# sorting values into highest to lowest base pay per year
avg_base_pay = avg_base_pay.sort_values('base-pay', ascending=False).reset_index()

avg_base_pay

,year,base-pay
0,2013,69039.50
1,2014,66651.85
2,2012,65500.11
3,2011,63601.39


In [138]:
# department analysis between fire and police depts in terms of total-pay-benefits
fire_dept = df[df['job-title'].str.contains("Fire Department", na=False)]

police_dept = df[df['job-title'].str.contains("Police Department", na=False)]

fire_dept.head(5)

,id,employee-name,job-title,base-pay,overtime-pay,other-pay,benefits,total-pay,total-pay-benefits,year
4,5,Patrick Gardner,"Deputy Chief Of Department,(Fire Department)",134401.60,9737.00,182234.59,0.0,326373.19,326373.19,2011
6,7,Alson Lee,"Battalion Chief, (Fire Department)",92492.01,89062.90,134426.14,0.0,315981.05,315981.05,2011
8,9,Michael Morris,"Battalion Chief, (Fire Department)",176932.64,86362.68,40132.23,0.0,303427.55,303427.55,2011
9,10,Joanne Hayes-White,"Chief Of Department, (Fire Department)",285262.00,0.00,17115.73,0.0,302377.73,302377.73,2011
10,11,Arthur Kenney,"Assistant Chief Of Department, (Fire Department)",194999.39,71344.88,33149.90,0.0,299494.17,299494.17,2011


In [139]:
police_dept.head(5)

,id,employee-name,job-title,base-pay,overtime-pay,other-pay,benefits,total-pay,total-pay-benefits,year
1,2,Gary Jimenez,Captain III (Police Department),155966.02,245131.88,137811.38,0.0,538909.28,538909.28,2011
2,3,Albert Pardini,Captain III (Police Department),212739.13,106088.18,16452.60,0.0,335279.91,335279.91,2011
11,12,Patricia Jackson,Captain III (Police Department),99722.00,87082.62,110804.30,0.0,297608.92,297608.92,2011
15,16,Richard Corriea,"Commander III, (Police Department)",198778.01,73478.20,13957.65,0.0,286213.86,286213.86,2011
28,29,Denise Schmitt,Deputy Chief III (Police Department),261717.60,0.00,2357.00,0.0,264074.60,264074.60,2011


#### Salary Structure & Distribution Analysis
- shows how salary or compensation is structured across roles and organizations.
- shows the average, median, min, and max total compensation per job title or department.

In [ ]:
salary_struct = df.groupby('job-title').agg({
    'job-title' : 'count',
    'total-pay-benefits' : ['mean', 'median', 'min', 'max', 'std']
})

# apply columns

header = [ 
    'job_count', 'avg_total_compensation', 'med_compensation', 'min_total_compensation',
    'max_total_compensation', 'std_total_compensation'
]

salary_struct.columns = header

salary_struct = salary_struct.fillna(0)

salary_struct = salary_struct.reset_index().round(2)
salary_struct

,job-title,job_count,avg_total_compensation,med_compensation,min_total_compensation,max_total_compensation,std_total_compensation
0,Account Clerk,349,58522.56,64746.53,280.46,91262.07,25602.12
1,Accountant,5,47429.27,60076.24,1148.40,65392.01,27137.33
2,Accountant I,8,88122.19,98191.27,28110.76,100245.90,24641.40
3,Accountant II,226,95086.02,98858.10,1799.96,112654.81,19239.86
4,Accountant III,278,107741.41,122634.03,213.22,143869.55,33413.98
...,...,...,...,...,...,...,...
1628,Worker'S Compensation Supervisor I,5,91020.73,92344.78,85277.39,95284.63,3839.05
1629,X-Ray Laboratory Aide,126,66051.31,70385.86,1017.07,129044.16,33717.38
1630,Youth Comm Advisor,4,60118.55,64871.28,24957.24,85774.39,27510.49
1631,"Youth Commission Advisor, Board Of Supervisors",1,53632.87,53632.87,53632.87,53632.87,0.00


#### Overtime & Extra Pay Dependence
- Analyze reliance on overtime and additional compensation.
- Compare job titles that has overtime pay > base pay, and those titles that does not have overtime pay

In [ ]:
overtime = df.groupby(['job-title', 'base-pay']).agg({
    'pver'
})

#### Time-Based Compensation Trends
- shows how the salaries evolve over the time per job title
- shows the year over year growth in terms of: average base pay & average total pay

In [141]:
salary_trends = df.groupby(['year', 'job-title']).agg({
    'base-pay' : 'mean',
    'total-pay' : 'mean'
})

salary_trends = salary_trends.reset_index().round(2)
salary_trends = salary_trends.sort_values('total-pay', ascending=False)

salary_trends.head()

,year,job-title,base-pay,total-pay
395,2011,General Manager-Metropolitan Transit Authority,199119.15,399211.28
3314,2014,Chief Investment Officer,257340.00,339653.70
2272,2013,Chief Of Police,319275.01,339282.07
2279,2013,"Chief, Fire Department",313686.01,336922.01
3316,2014,Chief Of Police,307450.04,326716.76
